# Bayesian MSD analysis
The purpose of ``bayesmsd`` is to provide a statistically rigorous method for fitting MSD curves. That is the intuition at least; what we are *really* doing is not actually a fit to the MSD curve. Instead we are working directly on the trajectory level:

+ Any given MSD curve (powerlaw or not) uniquely defines a steady state Gaussian process that exhibits this MSD;
+ given the Gaussian process, we can analytically calculate the likelihood of observing the given data;
+ this likelihood function then allows comparing different models (i.e. MSD curves) to each other and finding the one that best fits the data.
+ If we start out with a parametric family of MSD curves (e.g. a powerlaw: $\text{MSD}(\Delta t) = \Gamma(\Delta t)^\alpha$ would have two parameters $(\Gamma, \alpha)$), we can use Bayes' rule to calculate posterior distributions over the parameters. This provides us with rigorous estimates of uncertainty in the estimated values.

So at the end of the day, we find the **best fit Gaussian process to given data**. If the data can reasonably be assumed to be Gaussian, that seems to be the way to go of course; if the data is not Gaussian *per se*, we still get a best fit to second order (i.e. taking into account all two-point correlations).

Before diving deeper into the theoretical details, let's see how this works in practice.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import bayesmsd

## Generating some example data
We start by generating a simple example data set of normally diffusive trajectories: 20 trajectories, 100 frames each, in 2 spatial dimensions. For more info on accepted data formats, see below.

In [ ]:
np.random.seed(704724568)
data = np.cumsum(np.random.normal(size=(20, 100, 2)), axis=1)

# Visualize a few random ones
for i in [0, 3, 7, 10, 18]:
    plt.plot(data[i, :, 0])
plt.xlabel('time [frames]')
plt.ylabel('trajectories')
plt.show()

## Fitting a powerlaw MSD
A general powerlaw MSD has the form
\begin{equation}
\text{MSD}(\Delta t) = \Gamma (\Delta t)^\alpha\,.
\end{equation}
For our example data set we sampled the increments from a standard normal distribution, so we expect to see $\text{MSD}(\Delta t) = \Delta t$; i.e. both exponent ("$\alpha$") and prefactor ("anomalous diffusion constant") should be equal to one. Let's see what we find by fitting the data:

In [ ]:
fit = bayesmsd.lib.NPXFit(data, ss_order=1)
fit.parameters['log(σ²) (dim 0)'].fix_to = -np.inf # data is synthetic, so
fit.parameters['log(σ²) (dim 1)'].fix_to = -np.inf # no localization error

fitres = fit.run(show_progress=True)

print(f"fitted exponent:  {       fitres['params'][     'α (dim 0)'] :.2f}")
print(f"fitted prefactor: {np.exp(fitres['params']['log(Γ) (dim 0)']):.2f}")

This looks quite good; we can probably assume that the slight deviations from the theoretical values (1.00 for both exponent and prefactor) are statistical fluctuations due to finite data. But can we make a quantitative statement about that?

The Bayesian approach allows us to calculate credible intervals for the fit results; this is the job of the ``Profiler`` class:

In [ ]:
profiler = bayesmsd.Profiler(fit)
profiler.point_estimate = fitres

mci = profiler.find_MCI(show_progress=True)

m, ci = mci['α (dim 0)']
print(f"fitted exponent:  {m:.2f}, with 95% CI: [{ci[0]:.2f}, {ci[1]:.2f}]")

m, ci = mci['log(Γ) (dim 0)']
print(f"fitted prefactor: {np.exp(m):.2f}, with 95% CI: [{np.exp(ci[0]):.2f}, {np.exp(ci[1]):.2f}]")

Indeed, our theoretical expectations lie well within the credible interval of the fit. So we can conclude that standard normal diffusion is consistent with the trajectories in ``data``.

## Understanding what just happened
Let us understand the two code blocks above in some more detail.

 + We begin by creating an instance of the ``bayesmsd.lib.NPXFit`` class. ``NPX`` stands for "``N``oise + ``P``owerlaw + ``X``"; its core is the ``P``, i.e. fitting a powerlaw $\text{MSD} = \Gamma(\Delta t)^\alpha$ to the data.
 + The keyword argument ``ss_order=1`` tells the fit object that we are looking at increment stationary trajectories, as opposed to trajectories that are themselves stationary (in which case we would set ``ss_order=0``).
 + By default, ``NPXFit`` includes a ``N``oise floor (i.e. localization error) in its fit; since our data are synthetic diffusion—without any error added—we do not want the fit to have that free parameter. The second line
 
   ```py
   fit.parameters['log(σ²) (dim 0)'].fix_to = -np.inf
   ```
   
   tells our ``fit`` object to keep the parameter named ``'log(σ²) (dim 0)'`` (the log of the squared localization error; don't worry about the ``'(dim 0)'`` part for now, see below for that) fixed to the constant value ``-np.inf``, instead of trying to fit it to the data. The localization error that is assumed (along dimension 0, i.e. $x$) in the fit now has standard deviation $\sigma = \text{e}^{-\infty} = 0$. Thus, no error.
 + Line 3 achieves the same for the second dimension (remember that our data has two spatial dimensions)
 + The ``X`` of ``NPXFit`` can be used to extend the powerlaw with cubic splines at long times; it is disabled by default and we will not worry about it at this point.
 + Finally, we execute the fit by calling  ``fit.run()``.
 + For now we are just interested in the best-fit (MAP) parameter values, so we print them.

The fit leaves us with a point estimate for the parameters $\Gamma$ and $\alpha$. One of the true powers of the Bayesian approach is that we can now also calculate error bars ("credible intervals"). The ``Profiler`` class does this by tracing out the *profile posterior* for each parameter. Here are the details on the second code block above:

 + We initalize a new ``Profiler`` instance. The profiler should be understood as sitting downstream of the fit, i.e. it uses the ``Fit`` object to execute all its tasks.
 + Since we already ran the fit, we know the point estimate for the parameters and hand it to the profiler. If ``profiler.point_estimate`` remains unspecified, the profiler will run the fit itself.
 + ``find_MCI()`` is the main function of the ``Profiler``. ``MCI`` stands for "``M``oint estimate and ``C``redible ``I``nterval" (``M`` for point estimate is a historical accident), so this function returns point estimates and error bars for all the fit parameters.

That's it for this quickstart example. We saw how to initialize and run a powerlaw fit, and how to use the profiler to calculate error bars. These are the basic functionalities of the ``bayesmsd`` module.

---
TODO: write a "where to go next" paragraph and link to more topics

done:
 + changing parameter properties (bounds, linearization, fix_to)
 + loading data
 + parameter resolution order
 + generating trajectories (gp.generate)
 + plotting (using Fit.MSD)
 + show off ``bayesmsd.lib``
 + show off splines with AIC
 + motion blur

for users:
 
 
for advanced users:
 + Profiler rundown (just what happens in which order)
 + writing new fits
 + Fit.MinTarget

You can also implement fits to your own favorite model by subclassing ``bayesmsd.Fit`` (which provides all the core functionality). To do so, the only thing you need is a parametric expression for the MSD; as an example, ``bayesmsd.lib.RouseTwoLocusFit`` implements a fit to the following model:
\begin{equation}
\text{MSD}(\Delta t) = 2\Gamma\sqrt{\Delta t}\left(1 - \text{e}^{-\frac{\tau}{\Delta t}}\right) + 2J\,\text{erfc}\,\sqrt{\frac{\tau}{\Delta t}}\,,
\end{equation}
with parameters $\Gamma$ and $J$, and $\tau\equiv\frac{1}{\pi}\left(\frac{J}{\Gamma}\right)^2$.

---

## Epilogue: data formatting
How should data be formatted when handing them to a ``Fit`` object?

Above we used a big numpy array; while this was useful for synthetic data, it is not a particularly good format if trajectories can have different lengths. So what are the alternatives?

 + list of trajectories: a simple list of numpy arrays, each of shape ``(T, d)`` or ``(T,)`` where ``T`` is the number of frames and ``d`` the number of spatial dimensions. Missing frames can be indicated by ``np.nan``.
 + ``pandas.DataFrame``: a big data frame containing all individual data points can be formatted as follows: the column ``'particle'`` should contain a unique identifier for each trajectory; the column ``'frame'`` should contain integer frame numbers for all localizations; finally, coordinates for each localization should be in columns ``'x'``, ``'y'``, ``'z'`` (leave out ``'z'`` for 2D data, etc.)
 + ``noctiluca.TaggedSet``: under the hood, ``bayesmsd`` uses the [noctiluca](https://github.com/OpenTrajectoryAnalysis/noctiluca) library to format and handle single trajectories. ``TaggedSet`` is a data structure that can be used to dynamically aggregate different data sets and pick out specific subsets to work with. This is the natural data format for ``bayesmsd``; for details refer to the [noctiluca documentation](https://noctiluca.readthedocs.io/en/latest/index.html)
 + For more details see [noctiluca.util.userinput](https://noctiluca.readthedocs.io/en/latest/noctiluca.util.html#module-noctiluca.util.userinput), which is how ``bayesmsd`` deals with its input internally.

## Epilogue: ``ss_order``
**TL;DR:**

 + $\lim_{\Delta t\to\infty} \text{MSD}(\Delta t) < \infty \quad\Leftrightarrow\quad$ ``ss_order = 0``
 + $\lim_{\Delta t\to\infty} \text{MSD}(\Delta t) = \infty \quad\Leftrightarrow\quad$ ``ss_order = 1``
 
The parameter ``ss_order`` ("steady state order") controls an important distinction regarding the process that generates the data:

 + a steady state of order 0 means that the process generating the trajectories is itself in steady state. This is the case for example for a particle in a potential well (Ornstein-Uhlenbeck process), or any other process where the absolute coordinates carry some meaning
 + a steady state of order 1, in contrast, means that the generating process is *increment* stationary. This is the case for example for a standard random walk: the steps taken are drawn from a fixed distribution. Meanwhile, the absolute position of the walker carries no meaning (it will walk off to infinity in the long run either way).
 
Practically, the relevant question is: "does anything change, if I add 10 to all points in the trajectory?" (or, for that matter, if it is relevant "10" of which unit we mean there). If yes, that's ``ss_order = 0``. If no, use ``ss_order = 1``.